## Make Labelled videos with this workbook!

In [1]:
## importing all the necessary packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import cv2
import os

In [22]:
direc = os.getcwd()[0:-8] + 'videos'
df = pd.read_csv(direc + '\\csv_output\\AC_SOC2_3DLC_resnet50_social_behavior_allMay27shuffle1_250000.csv', header = [1, 2])
video = direc + '\\mp4s\\AC_SOC2_3.mp4'

In [2]:
def prepare_coordinates(direc):
    
    import numpy as np
    from shapely.geometry import Point
    from shapely.geometry.polygon import Polygon
    
    ## will want to start things off by loading in the coordinates drawn (this should be done beforehand)

    left_side = np.loadtxt(direc + 'left_side')
    x_chamber = np.loadtxt(direc + 'x_chamber')
    y_chamber = np.loadtxt(direc + 'y_chamber')
    right_side = np.loadtxt(direc + 'right_side')
    middle = np.loadtxt(direc + 'middle')
    
    ## calculate the padded interaction zone, that is 25 pixels larger in every direction (than the base of the chamber)

    x_outer = Polygon(Polygon(x_chamber).buffer(40.0).exterior).exterior.coords.xy
    y_outer = Polygon(Polygon(y_chamber).buffer(40.0).exterior).exterior.coords.xy
    x_center = np.mean(x_outer[0]), np.mean(x_outer[1])
    y_center = np.mean(y_outer[0]), np.mean(y_outer[1])
    
    x_zone = Polygon(Polygon(x_chamber).buffer(40.0).exterior)
    y_zone = Polygon(Polygon(y_chamber).buffer(40.0).exterior)
    
    possible_places = {'x_zone': x_zone, 'y_zone': y_zone, 'left_side': left_side, 'middle': middle, 'right_side': right_side}
    extra_coords = {'x_outer': x_outer, 'x_center': x_center, 'y_outer': y_outer, 'y_center': y_center, 'x_chamber': x_chamber, 'y_chamber': y_chamber}
    
    return possible_places, extra_coords

In [3]:
possible_places, extra_coords = prepare_coordinates(os.getcwd() + '\\coordinates\\')

In [4]:
possible_places

{'x_zone': <shapely.geometry.polygon.Polygon at 0x1ec7f0ede88>,
 'y_zone': <shapely.geometry.polygon.Polygon at 0x1ec7f0ee148>,
 'left_side': array([[242.36442529, 155.28589635],
        [254.98731642, 604.85501881],
        [481.22836509, 597.08708581],
        [473.46043209, 150.43093822]]),
 'middle': array([[481.22836509, 151.40192985],
        [495.79323947, 596.11609419],
        [709.41139701, 588.34816118],
        [696.78850589, 144.60498847]]),
 'right_side': array([[712.32437189, 149.4599466 ],
        [723.00527977, 593.20311931],
        [947.30434519, 583.49320306],
        [934.68145406, 135.86606384]])}

In [5]:
def check_coords(coords, possible_places = possible_places):

    import numpy as np
    from shapely.geometry import Point
    from shapely.geometry.polygon import Polygon
    
    x = []

    for i in range(len(list(possible_places.values()))):   
        pt = Point(coords)
        if isinstance(list(possible_places.values())[i], Polygon):
            polygon = list(possible_places.values())[i]
        else:
            polygon = Polygon(list(map(tuple, list(possible_places.values())[i])))
        x = np.append(x, polygon.contains(pt))

    return(x)

In [6]:
check_coords([350, 350])

array([1., 0., 1., 0., 0.])

In [7]:
def check_orientation_single(df, index_loc):
        
        orientation = 'not_oriented'
    
        dist_to_x = np.sqrt(((extra_coords['x_center'][0] - df['nose']['x'].loc[index_loc])**2) + ((extra_coords['x_center'][1] - df['nose']['y'].loc[index_loc])**2))
        dist_to_y = np.sqrt(((extra_coords['y_center'][0] - df['nose']['x'].loc[index_loc])**2) + ((extra_coords['y_center'][1] - df['nose']['y'].loc[index_loc])**2))
        
        if dist_to_x > dist_to_y:        
            distance_to_nose = np.sqrt(((extra_coords['y_center'][0] - df['nose']['x'].loc[index_loc])**2) + ((extra_coords['y_center'][1] - df['nose']['y'].loc[index_loc])**2))
            distance_to_l_ear = np.sqrt(((extra_coords['y_center'][0] - df['left ear']['x'].loc[index_loc])**2) + ((extra_coords['y_center'][1] - df['left ear']['y'].loc[index_loc])**2))
            distance_to_r_ear = np.sqrt(((extra_coords['y_center'][0] - df['right ear']['x'].loc[index_loc])**2) + ((extra_coords['y_center'][1] - df['right ear']['y'].loc[index_loc])**2))
        elif dist_to_x < dist_to_y:
            distance_to_nose = np.sqrt(((extra_coords['x_center'][0] - df['nose']['x'].loc[index_loc])**2) + ((extra_coords['x_center'][1] - df['nose']['y'].loc[index_loc])**2))
            distance_to_l_ear = np.sqrt(((extra_coords['x_center'][0] - df['left ear']['x'].loc[index_loc])**2) + ((extra_coords['x_center'][1] - df['left ear']['y'].loc[index_loc])**2))
            distance_to_r_ear = np.sqrt(((extra_coords['x_center'][0] - df['right ear']['x'].loc[index_loc])**2) + ((extra_coords['x_center'][1] - df['right ear']['y'].loc[index_loc])**2))

        if distance_to_nose == np.min([distance_to_nose, distance_to_l_ear, distance_to_r_ear]):
            orientation = 'oriented'
            
        return orientation

In [29]:
def calculate_investigation_times_single(df):
    
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from shapely.geometry import Point
    from shapely.geometry.polygon import Polygon
    import cv2
    from tqdm.notebook import tqdm
    
    bodyparts = np.unique(df.columns.get_level_values(0))[1:]

    arr = np.zeros(shape = (len(df), len(bodyparts), len(possible_places)))

    ### now we should check the coordinates of each bodypart in each frame
    print('Calculating Investigation Times: ')
    pbar = tqdm(total = len(df), leave = True)
    
    for row in range(len(df)):
        pbar.update()
        for j in range(len(bodyparts)):
            arr[row][j] = check_coords(df[bodyparts[j]][['x', 'y']].loc[row].values, possible_places)
    pbar.close()
            
    print('Array Constructed!')

    ### set which patterns mean x vs y investigation, only for the first three bodyparts (nose and ears, cuz we don't care about tail base yet)
    x_inv = np.array([[1., 0., 1., 0., 0.]])
    y_inv = np.array([[0., 1., 0., 0., 1.]])

    ### now we want to check each frame in our array, and create a frame_val array that holds info about where the mouse's head was detected
    z = -1
    frame_val = np.zeros(shape = len(arr), dtype = 'object')
    for frame in tqdm(range(len(arr))):
        z = z + 1
        comparison_x = arr[frame][0:1] == x_inv
        comparison_y = arr[frame][0:1] == y_inv

        if comparison_x.all() == True:
            if check_orientation_single(df, z) == 'oriented':
                frame_val[z] = 'X Investigation'
            elif check_orientation_single(df, z) == 'not_oriented':
                frame_val[z] = 'X Close'
        elif comparison_y.all() == True:
            if check_orientation_single(df, z) == 'oriented':
                frame_val[z] = 'Y Investigation'
            elif check_orientation_single(df, z) == 'not_oriented':
                frame_val[z] = 'Y Close'
        else:
            frame_val[z] = 'Somewhere else'
        
    print('Investigation Times Calculated!!')

    return frame_val

In [13]:
%%time

x = calculate_investigation_times_single(df)

Array Constructed



(array(['Somewhere else', 'X Close', 'X Investigation', 'Y Close',
       'Y Investigation'], dtype=object), array([10335,  2127,  1902,  1554,  1610], dtype=int64))
Wall time: 3min 41s


In [28]:
def export_labelled_frames(df, vname, frame_val, output_dir = 'labelled_frames', investigation = True):
    
    #### Should add head vector argument soon ####
    
    ## import all of the necessary packages
    import numpy as np
    import pandas as pd
    import cv2
    import os
    from tqdm.notebook import tqdm
    
    video = cv2.VideoCapture(vname)


    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    
    ## extract relevant meta information about the video
    frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    width  = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video.get(cv2.CAP_PROP_FPS))
    
    ## while loop through every frame of the video and label each frame
    success, image = video.read()
    count = 0
    pbar = tqdm(total = len(df)+1)
    while success:
        nose_coords = (int(df['nose']['x'].loc[count]), int(df['nose']['y'].loc[count]))
        midpoint_coords = (int((df['right ear']['x'].loc[count] + df['left ear']['x'].loc[count]) / 2) , int((df['right ear']['y'].loc[count] + df['left ear']['y'].loc[count]) / 2))
        if frame_val[count] == 'Somewhere else':
            color = (0, 0, 255)
        if frame_val[count] == 'X Close':
            color = (0, 0, 255)
        if frame_val[count] == 'Y Close':
            color = (0, 0, 255)
        if frame_val[count] == 'X Investigation':
            color = (0, 255, 0)
        if frame_val[count] == 'Y Investigation':
            color = (0, 255, 0)
        pbar.update(1)
        image = cv2.line(image, nose_coords, midpoint_coords, color, 4)
        cv2.imwrite(output_dir + '\\frame_' + str(count) + '.png', image)
        success,image = video.read()
        count += 1
    pbar.close()

In [30]:
direc = os.getcwd()[0:-8] + 'videos'

videos = os.listdir(direc + '\\mp4s')

for video in videos:
    df = pd.read_csv(direc + '\\csv_output\\' + video[0:-4] + 'DLC_resnet50_social_behavior_allMay27shuffle1_250000.csv', header = [1, 2])
    video_name = direc + '\\mp4s\\' + video
    z = calculate_investigation_times_single(df)
    export_labelled_frames(df, vname = video_name, frame_val = z, output_dir = 'E:\\DATA\\' + video[0:-4] + '_labelled_frames_correct_green_red')
    print(video + 'Frames Finished!')

Calculating Investigation Times: 



Array Constructed!



Investigation Times Calculated!!



AC_SOC2_1.mp4Frames Finished!
Calculating Investigation Times: 



Array Constructed!


...


AC_SOC4_11.mp4Frames Finished!
Calculating Investigation Times: 



Array Constructed!



Investigation Times Calculated!!



AC_SOC4_12.mp4Frames Finished!
Calculating Investigation Times: 


KeyboardInterrupt: 

In [25]:
export_labelled_frames(df, vname = video, frame_val = x, output_dir = 'E:\\DATA\\' + 'AC_SOC2_3' + '_labelled_frames_correct_green_red')

KeyboardInterrupt: 

## Once you have all of the frames exported, you can stitch them back together with [virtualdub](https://www.cmiss.org/cmgui/wiki/CreatingAnAVIFromASeriesOfImagesWindows)